# Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
production = pd.read_csv('processed data/production.csv', encoding='ISO-8859-1').drop(['Unnamed: 0'], axis = 1)

balance_historic = pd.read_csv('processed data/balance_historic.csv', encoding='ISO-8859-1').drop(['Unnamed: 0'], axis = 1)

balance = pd.read_csv('processed data/balance.csv', encoding='ISO-8859-1').drop(['Unnamed: 0'], axis = 1)

countries = pd.read_csv('processed data/countries.csv', encoding='ISO-8859-1').drop(['Unnamed: 0'], axis = 1)

print('production shape: ', production.shape)
print('balance shape: ', balance.shape)
print('balance_historic shape: ', balance_historic.shape)

display(production.head())
display(balance_historic.head())
countries.head()

production shape:  (4837117, 7)
balance shape:  (1958440, 7)
balance_historic shape:  (8119335, 7)


,area,item_code_cpc,item,element,unit,year,value
0,Afghanistan,'01371,"Almonds, in shell",Area harvested,ha,1961,0.0
1,Afghanistan,'01371,"Almonds, in shell",Yield,hg/ha,1961,0.0
2,Afghanistan,'01371,"Almonds, in shell",Production,tonnes,1961,0.0
3,Afghanistan,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",Area harvested,ha,1961,0.0
4,Afghanistan,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",Yield,hg/ha,1961,0.0


,area,item_code_cpc,item,element,unit,year,value
0,Afghanistan,'S2501,Population,Total Population - Both sexes,1000 persons,1961,8954.00
1,Afghanistan,'S2901,Grand Total,Food supply,kcal/capita/day,1961,2999.00
2,Afghanistan,'S2901,Grand Total,Protein supply quantity,g/capita/day,1961,84.91
3,Afghanistan,'S2901,Grand Total,Fat supply quantity,g/capita/day,1961,37.51
4,Afghanistan,'S2903,Vegetal Products,Food supply,kcal/capita/day,1961,2752.00


,region_name,sub_region_name,country_or_area,iso_alpha3_code,least_developed_countries_ldc
0,Africa,Northern Africa,Algeria,DZA,0
1,Africa,Northern Africa,Egypt,EGY,0
2,Africa,Northern Africa,Libya,LBY,0
3,Africa,Northern Africa,Morocco,MAR,0
4,Africa,Northern Africa,Sudan,SDN,x


In [25]:
production = pd.merge(left = production,
                                 right = countries,
                                 how = 'inner', 
                                 left_on = "area", 
                                 right_on= "country_or_area").drop('country_or_area', axis=1)

balance = pd.merge(left = balance,
                                 right = countries,
                                 how = 'inner', 
                                 left_on = "area", 
                                 right_on= "country_or_area").drop('country_or_area', axis=1)

balance_historic = pd.merge(left = balance_historic,
                                 right = countries,
                                 how = 'inner', 
                                 left_on = "area", 
                                 right_on= "country_or_area").drop('country_or_area', axis=1)

production.head()

,area,item_code_cpc,item,element,unit,year,value,region_name,sub_region_name,iso_alpha3_code,least_developed_countries_ldc
0,Afghanistan,'01371,"Almonds, in shell",Area harvested,ha,1961,0.0,Asia,Southern Asia,AFG,x
1,Afghanistan,'01371,"Almonds, in shell",Yield,hg/ha,1961,0.0,Asia,Southern Asia,AFG,x
2,Afghanistan,'01371,"Almonds, in shell",Production,tonnes,1961,0.0,Asia,Southern Asia,AFG,x
3,Afghanistan,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",Area harvested,ha,1961,0.0,Asia,Southern Asia,AFG,x
4,Afghanistan,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",Yield,hg/ha,1961,0.0,Asia,Southern Asia,AFG,x


In the production and balance datasets, useful *features* are buried within the element column. I will combine these with the unit column and turn into columns themselves.

Further, population data in the balance datsets is an area-level parameter and should be a column itself. We will use it to make sense of the balance features many of which are scaled by population (eg. per capita).

In [26]:
balance['element_unit'] = balance[['element', 'unit']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
balance_historic['element_unit'] = balance_historic[['element', 'unit']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
production['element_unit'] = production[['element', 'unit']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [27]:
def restructure_pivot(df):
       
    df_pivot = df.pivot_table(index = ['sub_region_name','area','iso_alpha3_code','least_developed_countries_ldc','year','item_code_cpc','item'], columns=['element_unit'], aggfunc = {'value':'sum'})
    
    col_list = []
    for col in df_pivot.columns:
        col_list.append(col[1])

    df_pivot.columns = col_list

    df_pivot = df_pivot.reset_index()
    
    return df_pivot

In [37]:
balance_pivot = restructure_pivot(balance)
balance_pivot.head()

,sub_region_name,area,iso_alpha3_code,least_developed_countries_ldc,year,item_code_cpc,item,Export Quantity_1000 tonnes,Fat supply quantity _g/capita/day,Food supply _kcal/capita/day,Food_1000 tonnes,Import Quantity_1000 tonnes,Losses_1000 tonnes,Production_1000 tonnes,Protein supply quantity _g/capita/day,Total Population - Both sexes_1000 persons
0,Australia and New Zealand,Australia,AUS,0,2010,'S2501,Population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22155.0
1,Australia and New Zealand,Australia,AUS,0,2010,'S2511,Wheat and products,16143.0,2.23,539.91,1510.0,170.0,218.0,21834.0,16.96,NaN
2,Australia and New Zealand,Australia,AUS,0,2010,'S2513,Barley and products,4789.0,0.00,0.00,0.0,7.0,33.0,7865.0,0.00,NaN
3,Australia and New Zealand,Australia,AUS,0,2010,'S2514,Maize and products,11.0,0.12,43.70,100.0,16.0,2.0,328.0,0.95,NaN
4,Australia and New Zealand,Australia,AUS,0,2010,'S2515,Rye and products,0.0,0.00,0.39,1.0,1.0,1.0,29.0,0.01,NaN


In [38]:
def population(df):
    pop = balance_p[['area','year','Total Population - Both sexes_1000 persons']].copy()
    pop = pop.dropna()
    
    df = df.drop(['Total Population - Both sexes_1000 persons'],axis=1)
    df.drop(df[df['item'] == 'Population'].index, inplace = True)
    
    df = pd.merge(df, pop,  how='inner', left_on=['area','year'], right_on = ['area','year'])
    
    return df

In [39]:
balance_pivot = population(balance_pivot)
balance_pivot.head()

,sub_region_name,area,iso_alpha3_code,least_developed_countries_ldc,year,item_code_cpc,item,Export Quantity_1000 tonnes,Fat supply quantity _g/capita/day,Food supply _kcal/capita/day,Food_1000 tonnes,Import Quantity_1000 tonnes,Losses_1000 tonnes,Production_1000 tonnes,Protein supply quantity _g/capita/day,Total Population - Both sexes_1000 persons
0,Australia and New Zealand,Australia,AUS,0,2010,'S2511,Wheat and products,16143.0,2.23,539.91,1510.0,170.0,218.0,21834.0,16.96,22155.0
1,Australia and New Zealand,Australia,AUS,0,2010,'S2513,Barley and products,4789.0,0.00,0.00,0.0,7.0,33.0,7865.0,0.00,22155.0
2,Australia and New Zealand,Australia,AUS,0,2010,'S2514,Maize and products,11.0,0.12,43.70,100.0,16.0,2.0,328.0,0.95,22155.0
3,Australia and New Zealand,Australia,AUS,0,2010,'S2515,Rye and products,0.0,0.00,0.39,1.0,1.0,1.0,29.0,0.01,22155.0
4,Australia and New Zealand,Australia,AUS,0,2010,'S2516,Oats,347.0,0.01,0.76,3.0,1.0,23.0,1162.0,0.03,22155.0


In [40]:
balance_pivot.isna().sum()

sub_region_name                                    0
area                                               0
iso_alpha3_code                                    0
least_developed_countries_ldc                      0
year                                               0
item_code_cpc                                      0
item                                               0
Export Quantity_1000 tonnes                    27676
Fat supply quantity _g/capita/day              15829
Food supply _kcal/capita/day                   16027
Food_1000 tonnes                               20801
Import Quantity_1000 tonnes                     8030
Losses_1000 tonnes                            132902
Production_1000 tonnes                         70268
Protein supply quantity _g/capita/day          15785
Total Population - Both sexes_1000 persons         0
dtype: int64

I will fill all nulls with 0.

In [41]:
balance_pivot = balance_pivot.fillna(0)

We will need additional columns for analysis:

In [42]:
balance_pivot.columns

Index(['sub_region_name', 'area', 'iso_alpha3_code',
       'least_developed_countries_ldc', 'year', 'item_code_cpc', 'item',
       'Export Quantity_1000 tonnes', 'Fat supply quantity _g/capita/day',
       'Food supply _kcal/capita/day', 'Food_1000 tonnes',
       'Import Quantity_1000 tonnes', 'Losses_1000 tonnes',
       'Production_1000 tonnes', 'Protein supply quantity _g/capita/day',
       'Total Population - Both sexes_1000 persons'],
      dtype='object')

In [43]:
def new_columns(df):
    df['food_supply_kcal_per_day'] = 1000 * df['Food supply _kcal/capita/day'] * df['Total Population - Both sexes_1000 persons']
    df['kcal_per_1000tonnes'] = 365 * df['food_supply_kcal_per_day'] / df['Food_1000 tonnes']
    df['import_multiple'] = df['Import Quantity_1000 tonnes'] / df['Food_1000 tonnes']
    df['export_multiple'] = df['Export Quantity_1000 tonnes'] / df['Food_1000 tonnes']
    df['loss_multiple'] = df['Losses_1000 tonnes'] / df['Food_1000 tonnes']
    df['losses_kcal_per_day'] = df['loss_multiple'] * df['food_supply_kcal_per_day']
    df['fat_supply_g_per_day'] = df['Fat supply quantity _g/capita/day'] * df['Total Population - Both sexes_1000 persons']
    df['protein_supply_g_per_day'] = df['Protein supply quantity _g/capita/day'] * df['Total Population - Both sexes_1000 persons']
    return df

In [44]:
balance_final = new_columns(balance_pivot)
balance_final

,sub_region_name,area,iso_alpha3_code,least_developed_countries_ldc,year,item_code_cpc,item,Export Quantity_1000 tonnes,Fat supply quantity _g/capita/day,Food supply _kcal/capita/day,...,Protein supply quantity _g/capita/day,Total Population - Both sexes_1000 persons,food_supply_kcal_per_day,kcal_per_1000tonnes,import_multiple,export_multiple,loss_multiple,losses_kcal_per_day,fat_supply_g_per_day,protein_supply_g_per_day
0,Australia and New Zealand,Australia,AUS,0,2010,'S2511,Wheat and products,16143.00,2.23,539.91,...,16.96,22155.00,1.196171e+10,2.891406e+09,0.112583,10.690728,0.144371,1.726922e+09,49405.6500,375748.8000
1,Australia and New Zealand,Australia,AUS,0,2010,'S2513,Barley and products,4789.00,0.00,0.00,...,0.00,22155.00,0.000000e+00,NaN,inf,inf,inf,NaN,0.0000,0.0000
2,Australia and New Zealand,Australia,AUS,0,2010,'S2514,Maize and products,11.00,0.12,43.70,...,0.95,22155.00,9.681735e+08,3.533833e+09,0.160000,0.110000,0.020000,1.936347e+07,2658.6000,21047.2500
3,Australia and New Zealand,Australia,AUS,0,2010,'S2515,Rye and products,0.00,0.00,0.39,...,0.01,22155.00,8.640450e+06,3.153764e+09,1.000000,0.000000,1.000000,8.640450e+06,0.0000,221.5500
4,Australia and New Zealand,Australia,AUS,0,2010,'S2516,Oats,347.00,0.01,0.76,...,0.03,22155.00,1.683780e+07,2.048599e+09,0.333333,115.666667,7.666667,1.290898e+08,221.5500,664.6500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234680,Western Europe,Switzerland,CHE,0,2020,'S2946,Animal fats,11.00,17.89,159.00,...,0.14,8654.62,1.376085e+09,6.975984e+09,0.361111,0.152778,0.027778,3.822457e+07,154831.1518,1211.6468
234681,Western Europe,Switzerland,CHE,0,2020,'S2948,Milk - Excluding Butter,589.00,30.74,468.00,...,29.86,8654.62,4.050362e+09,5.831882e+08,0.234714,0.232347,0.000000,0.000000e+00,266043.0188,258426.9532
234682,Western Europe,Switzerland,CHE,0,2020,'S2949,Eggs,0.00,2.98,42.00,...,3.43,8654.62,3.634940e+08,1.426616e+09,0.408602,0.000000,0.064516,2.345123e+07,25790.7676,29685.3466
234683,Western Europe,Switzerland,CHE,0,2020,'S2960,"Fish, Seafood",0.82,1.19,29.00,...,4.15,8654.62,2.509840e+08,6.617724e+08,0.974500,0.005924,0.000000,0.000000e+00,10298.9978,35916.6730


I will just consider the aggregated food groups for the initial national analysis.

In [47]:
pd.set_option('display.max_columns', None)

In [ ]:
balance_national = balance_final[balance_final['item']]

In [48]:
balance_filtered = balance_final[balance_final['item_code_cpc'].str[:4] == "'S29"]
balance_filtered

,sub_region_name,area,iso_alpha3_code,least_developed_countries_ldc,year,item_code_cpc,item,Export Quantity_1000 tonnes,Fat supply quantity _g/capita/day,Food supply _kcal/capita/day,Food_1000 tonnes,Import Quantity_1000 tonnes,Losses_1000 tonnes,Production_1000 tonnes,Protein supply quantity _g/capita/day,Total Population - Both sexes_1000 persons,food_supply_kcal_per_day,kcal_per_1000tonnes,import_multiple,export_multiple,loss_multiple,losses_kcal_per_day,fat_supply_g_per_day,protein_supply_g_per_day
95,Australia and New Zealand,Australia,AUS,0,2010,'S2901,Grand Total,0.00,151.27,3410.0,0.00,0.0,0.0,0.00,110.87,22155.00,7.554855e+10,inf,NaN,NaN,NaN,NaN,3.351387e+06,2.456325e+06
96,Australia and New Zealand,Australia,AUS,0,2010,'S2903,Vegetal Products,0.00,78.17,2363.0,0.00,0.0,0.0,0.00,36.81,22155.00,5.235226e+10,inf,NaN,NaN,NaN,NaN,1.731856e+06,8.155256e+05
97,Australia and New Zealand,Australia,AUS,0,2010,'S2905,Cereals - Excluding Beer,21707.00,2.62,698.0,2014.00,518.0,382.0,33495.00,20.21,22155.00,1.546419e+10,2.802596e+09,0.257200,10.778054,0.189672,2.933128e+09,5.804610e+04,4.477526e+05
98,Australia and New Zealand,Australia,AUS,0,2010,'S2907,Starchy Roots,48.00,0.15,95.0,1253.00,118.0,29.0,1328.00,2.43,22155.00,2.104725e+09,6.131082e+08,0.094174,0.038308,0.023144,4.871271e+07,3.323250e+03,5.383665e+04
99,Australia and New Zealand,Australia,AUS,0,2010,'S2908,Sugar Crops,0.00,0.00,0.0,0.00,0.0,1680.0,31235.00,0.00,22155.00,0.000000e+00,NaN,NaN,NaN,inf,NaN,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234680,Western Europe,Switzerland,CHE,0,2020,'S2946,Animal fats,11.00,17.89,159.0,72.00,26.0,2.0,105.00,0.14,8654.62,1.376085e+09,6.975984e+09,0.361111,0.152778,0.027778,3.822457e+07,1.548312e+05,1.211647e+03
234681,Western Europe,Switzerland,CHE,0,2020,'S2948,Milk - Excluding Butter,589.00,30.74,468.0,2535.00,595.0,0.0,3832.00,29.86,8654.62,4.050362e+09,5.831882e+08,0.234714,0.232347,0.000000,0.000000e+00,2.660430e+05,2.584270e+05
234682,Western Europe,Switzerland,CHE,0,2020,'S2949,Eggs,0.00,2.98,42.0,93.00,38.0,6.0,66.00,3.43,8654.62,3.634940e+08,1.426616e+09,0.408602,0.000000,0.064516,2.345123e+07,2.579077e+04,2.968535e+04
234683,Western Europe,Switzerland,CHE,0,2020,'S2960,"Fish, Seafood",0.82,1.19,29.0,138.43,134.9,0.0,3.43,4.15,8654.62,2.509840e+08,6.617724e+08,0.974500,0.005924,0.000000,0.000000e+00,1.029900e+04,3.591667e+04


In [ ]:
production_overview = production_data.groupby(['area','element']).agg({'value':sum})
production_overview

In [ ]:
df = pd.DataFrame(production_data.pivot_table(index = ['area'], columns=['element'], aggfunc = {'value':'sum'}))
df.columns

In [ ]:
col_list = []
for col in df.columns:
    col_list.append(col[1])
col_list

df.columns = col_list

df = df[['Area harvested','Producing Animals/Slaughtered','Production']].reset_index()
df

In [ ]:
sns.scatterplot(data = df, x = 'Area harvested', y = 'Production')

In [ ]:
balance_hist[(balance_hist['element']=='Food supply (kcal/capita/day)')&(balance_hist['item']=='Grand Total')].value.describe().T

In [ ]:
balance_hist_pivot = balance_hist[balance_hist['item'].isin(['Grand Total','Population','Vegetal Products','Animal Products'])==True].pivot_table(index = ['sub-region_name','area','year'], columns=['element'], aggfunc = {'value':'sum'})
balance_hist_pivot.columns

In [ ]:
col_list = []
for col in balance_hist_pivot.columns:
    col_list.append(col[1])
col_list

balance_hist_pivot.columns = col_list

balance_hist_pivot = balance_hist_pivot[['Food supply (kcal/capita/day)','Total Population - Both sexes']].reset_index()
balance_hist_pivot

In [ ]:
balance_hist_pivot['Food supply (kcal/day)'] = 1000*balance_hist_pivot['Food supply (kcal/capita/day)']*balance_hist_pivot['Total Population - Both sexes']
balance_hist_pivot['Total population'] = 1000*balance_hist_pivot['Total Population - Both sexes']

balance_hist_pivot

In [ ]:
summary = balance_hist_pivot.groupby(['sub-region_name','year']).agg({'Food supply (kcal/day)':sum, 'Total population':sum})
summary_df = pd.DataFrame(summary).reset_index()
summary_df['Food supply (kcal/capita/day)'] = summary_df['Food supply (kcal/day)']/summary_df['Total population']
summary_df

In [ ]:
sns.lineplot(data = summary_df, x = 'year', y = 'Food supply (kcal/capita/day)', hue="sub-region_name")

In [ ]:
summary_df.describe().T

In [ ]:
# since 1980 every sub-region has had an average food supply of 2,000+ kcal/day
# all are increasing

In [ ]:
balance_hist['least_developed_countries_(ldc)'].value_counts(dropna=False)

In [ ]:
ldc = balance_hist[(balance_hist['least_developed_countries_(ldc)']=='x')&
                   (balance_hist['element']=='Food supply (kcal/capita/day)')&
                   (balance_hist['item']=='Grand Total')]
sns.lineplot(data = ldc, x = 'year', y = 'value', hue="area")

In [ ]:
ldc.describe().T

In [ ]:
balance_hist.head()